In [1]:
import os, sys
# import numpy as np
from functools import partial
import autograd.numpy as np
from autograd import jacobian, grad
from scipy.stats import multivariate_normal

## <center>                                           Add module paths

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MomentMatching.StateModels import GaussianState
from MomentMatching.baseMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform
from MomentMatching.auto_grad import logpdf
from MomentMatching.KalmanFilter import KalmanFilterSmoother

In [3]:
x = np.array([2.0, 0.1])

### <center> Define State Distribution

In [31]:
xx_mean = np.array([1.0, 0.0], dtype=float)
xx_sigma = np.array([[1, 0], [0, 1]], dtype=float)
state_distribution = GaussianState(xx_mean, xx_sigma)


### <center>  Define a linear function 

In [5]:
A = np.array([[0.3, 0],
              [0, 0.1]], dtype=float)
B = np.array([0.0, 0.0], dtype=float)

In [6]:
def measurement_function(x, A=A, B=B):
    return np.dot(A, x) + B#[:, np.newaxis]

### <center> Define Taylor transform TT

In [7]:
TT = TaylorTransform(dimension_of_state=2)

In [8]:
TT.numerical_jacobian(f=measurement_function, x=xx_mean)

array([[ 0.3,  0. ],
       [ 0. ,  0.1]])

### <center> 

``` python 
def measurement_update(self, node, measurement):

    assert isinstance(node, TimeSeriesNodeForEP)

    measurement_cavity_distribution = node.marginal / node.measurement_factor  # q(x_t) / q_up(x_t)

    new_node = node

    new_node.marginal = self.moment_matching(self.measurement, measurement_cavity_distribution, self.R, measurement)

    new_node.measurement_factor = new_node.marginal / measurement_cavity_distribution

    return new_node


```

In [9]:
meas_jac = jacobian(measurement_function)
meas_jac(xx_mean)

array([[ 0.3,  0. ],
       [ 0. ,  0.1]])

In [10]:
factor_mean = np.array([0.0, 0.0], dtype=float)
factor_sigma = 9999 * np.array([[1, 0], [0, 1]], dtype=float)
measurement_factor = GaussianState(factor_mean, factor_sigma)
measurement = np.array([2.0, 0.0], dtype=float)

In [11]:
def data_likelihood(f, mean, cov, measurement ):
    
    meanz = f(mean)   # z = f(x)
    linear_C = jacobian(f)(mean)  # linear factor A 
    covz = linear_C @ cov @ linear_C.T #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
    logZi = logpdf(measurement,meanz, covz)
    return logZi

In [12]:
# logZi_func = logpdf
# dlogZidMz_func = jacobian(logpdf, argnum=1)
# dlogZidSz_func = jacobian(logpdf, argnum=2)

# logZi = logZi_func(x=measurement, mean=mz, cov=sz)
# dlogZidMz = dlogZidMz_func(measurement, mz, sz)
# dlogZidSz = dlogZidSz_func(measurement, mz, sz) 

In [13]:
def moment_matching_gaussian(mxni, Sxni, dlogZidMz, dlogZidSz):
    mx = mxni + np.dot(Sxni, dlogZidMz.T)
    return mx

In [28]:
def ep_measurement_update(self, measurement_function, state_distribution,  z_measurement):
    measurement_cavity = state_distribution / measurement_factor
    logZi = data_likelihood(measurement_function, measurement_cavity.mean, measurement_cavity.cov, measurement)
    dlogZidMz = jacobian(data_likelihood, argnum=1)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    dlogZidSz = jacobian(data_likelihood, argnum=2)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    mx = measurement_cavity.mean + np.dot(measurement_cavity.cov, dlogZidMz.T)
    print(dlogZidMz)
    print(dlogZidSz.shape)
    vx = measurement_cavity.cov + measurement_cavity.cov @ (2*dlogZidSz - np.dot(dlogZidMz.T, dlogZidMz) ) @ measurement_cavity.cov
    return mx, vx
    

In [29]:
def measurement_update(self, measurement_function, state_distribution, z_measurement):
#     pred_mean, pred_cov, _ = self.predict(self.transition, x_distribution, t)

#     # Add transition Noise Q_t
#     pred_cov = pred_cov + self.system_model.Q.cov
#     pred_distribution = GaussianState(pred_mean, pred_cov)

    z_mean, z_cov, xz_cross_cov = self.predict(measurement_function, state_distribution)
    # Add measurement Noise R_t
    z_cov = z_cov #+ self.system_model.R.cov

    K = np.matmul(xz_cross_cov, np.linalg.inv(z_cov))
    print(f'kalman gain K is {K}')
    corrected_mean = state_distribution.mean + np.dot(K, (z_measurement - z_mean))  # equation 15  in Marc's ACC paper
    corrected_cov = state_distribution.cov - np.dot(K, np.transpose(xz_cross_cov))
    print(f'corrected cov is {corrected_cov}')
    filtered_distribution = GaussianState(corrected_mean, corrected_cov)
    return filtered_distribution

In [33]:
corrected_ans = measurement_update(TT, measurement_function, state_distribution, measurement)

kalman gain K is [[  3.33333333   0.        ]
 [  0.          10.        ]]
corrected cov is [[ 0.  0.]
 [ 0.  0.]]


In [34]:
corrected_ans.mean

array([ 6.66666667,  0.        ])

In [18]:
corrected_ans.cov

array([[  1.11022302e-16,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00]])

In [32]:
ep_measurement_update(0, measurement_function, state_distribution, measurement)

[ 5.66599993  0.        ]
(2, 2)


(array([ 6.66666667,  0.        ]),
 array([[  2.22044605e-15,  -3.21099776e+01],
        [ -3.21099776e+01,  -3.21099776e+01]]))

In [35]:
z_mean, z_cov, xz_cross_cov = TT.predict(measurement_function, state_distribution)

In [37]:
state_distribution.cov @ A

array([[ 0.3,  0. ],
       [ 0. ,  0.1]])

In [40]:
xz_cross_cov @ np.linalg.pinv(z_cov) @ xz_cross_cov.T

array([[ 1.,  0.],
       [ 0.,  1.]])

In [20]:
TT.numerical_jacobian(dummy, sz)

NameError: name 'dummy' is not defined

In [ ]:
jac = jacobian(dummy)

In [ ]:
jac(sz)